In [1]:
# instalacao das biblbiotecas
!pip install pandas
!pip install boto3

In [2]:
# importacao de bibliotecas
import numpy as np
import pandas as pd
import boto3
from io import StringIO 
from io import BytesIO

In [3]:
# conexao com MinIo
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='rwGPDEWtGtCIQnKe',
    aws_secret_access_key='dQWHX0cNzHFM83UZOiKlsPdpNK4PRMtm',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [4]:
# lendo o dataset estados.csv
df_estados = pd.read_csv('exercicios/municipios-estados/csv/estados.csv')

In [5]:
# lendo o dataset municipioss.csv
df_municipios = pd.read_csv('exercicios/municipios-estados/csv/municipios.csv')

In [6]:
# juncao dos 2 datasets(estados e municipios)
df = df_municipios.merge(df_estados[['uf', 'codigo_uf']], on='codigo_uf')
df

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO
...,...,...,...,...,...,...,...,...,...,...
5565,3306107,Valença,-22.2445,-43.7129,0,33,5921,24,America/Sao_Paulo,RJ
5566,3306156,Varre-Sai,-20.9276,-41.8701,0,33,2917,22,America/Sao_Paulo,RJ
5567,3306206,Vassouras,-22.4059,-43.6686,0,33,5923,24,America/Sao_Paulo,RJ
5568,3306305,Volta Redonda,-22.5202,-44.0996,0,33,5925,24,America/Sao_Paulo,RJ


In [7]:
df = df.rename(columns={'nome': 'cidade'})

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   cidade        5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   capital       5570 non-null   int64  
 5   codigo_uf     5570 non-null   int64  
 6   siafi_id      5570 non-null   int64  
 7   ddd           5570 non-null   int64  
 8   fuso_horario  5570 non-null   object 
 9   uf            5570 non-null   object 
dtypes: float64(2), int64(5), object(3)
memory usage: 478.7+ KB


In [9]:
uf = df['uf'].drop_duplicates()
uf

0       GO
246     MG
1099    PA
1243    CE
1427    BA
1844    PR
2243    SC
2538    PE
2723    TO
2862    MA
3079    RN
3246    PI
3470    RS
3967    MT
4108    AC
4130    SP
4775    ES
4853    PB
5076    AL
5178    MS
5257    RO
5309    RR
5324    AM
5386    AP
5402    SE
5477    RJ
5569    DF
Name: uf, dtype: object

In [10]:
# iterar dataframe por uf
# para cada linha filtrada do dataframe enviar para para a respectiva pasta com o nome da uf no bucket
for uf in df['uf'].unique():
    df_tmp = df.loc[ df['uf'] == uf ]
    
    #armazenando a variavel df_temp em buffer
    csv_buffer = StringIO()
    df_tmp.to_csv(csv_buffer)
    
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-06', Key=f'municipio_estado/{uf}/cidades.csv')